In [1]:
import os
import numpy as np
import pandas as pd
import sys
import pickle
# change this to include MNM_mcnb.py from MAC-POSTS
sys.path.append("/home/alanpi/Desktop/MAC-POSTS/side_project/network_builder")
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')
from matplotlib import colors
import matplotlib
import matplotlib.dates as mdates
import datetime
import seaborn as sns
import math
from scipy import stats
from sklearn.metrics import r2_score
import random

In [2]:
import MNMAPI
from MNM_mcnb import *
from cartruckmodels import *

## Links

In [3]:
''' Refer to the slides or report for the locations of intersection A to F '''
MR_link_ID_list = [1007186,7186, # A <-> B
                   1007174,7174, # B <-> C
                   7173,1007173, # C <-> D
                   7177,1003858, # D  -> E
                   3858,1007177, # E  -> D
                   3857,1003857, # E <-> F
                   4150,3863,    # A
                   7167,         # B
                   1022916,      # C
                   1007176,      # D
                   3832,1003862,1022923 # F
                  ]

link_map = {1007186:"A -> B", 7186:"A <- B", 1007174:"B -> C", 7174:"B <- C", 7173:"C -> D", 1007173:"C <- D",
            7177:"D -> E", 1003858:"D -> E", 3858:"E -> D", 1007177:"E -> D", 3857:"E -> F", 1003857:"E <- F"}

## Scenarios

In [4]:
''' Select only one input data folder from the scenarios below: '''

# Scenario 0: No traffic management or control scheme; No population and industrial/freight traffic growth from 2016 
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_0'

# # Scenario 1: West Carson street extension, ‘truck only’
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_1'

# # Scenario 2: West Carson street extension, ‘car and truck’
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_2'

# # Scenario 3: Change of development plans at the P&LE site 
data_folder = '../data/input_files_MckeesRocks_SPC_scenario_3'

# # Scenario 4: No left truck turns on Angelina Avenue
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_4'

# # Scenario 5: Scenario 1 & 4 together
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_5'

# # Scenario 6: 20% increase of population (20% increase in car use) 
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_6'

# # Scenario 7: 10% increase in industrial/freight traffic
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_7'

## Simulation

In [5]:
for repeat_time in range(3):
    print("------------------------------------{0}-------------------------------------------".format(repeat_time+1))
    nb = MNM_network_builder()
    nb.load_from_folder(data_folder)
    mcdta = Cartruck_Simulator(nb)
    mcdta.run_simulation(data_folder, MR_link_ID_list)
    
    
    # Travel time between nodes
    mid_intervals = np.array([i * 180 + 90 for i in range(24)])
#     mid_intervals = np.array([10])

    car_link_tt = mcdta.get_link_car_tt(mid_intervals)
    truck_link_tt = mcdta.get_link_truck_tt(mid_intervals)

    s_car = ''
    k = 6
    for i in range(6):
        s_car += str(round(np.mean(car_link_tt[i]),2)*k) + ' '
    s_car += str(round(np.mean(car_link_tt[6]) + np.mean(car_link_tt[7]),2)*k) + ' '
    s_car += str(round(np.mean(car_link_tt[8]) + np.mean(car_link_tt[9]),2)*k) + ' '
    s_car += str(round(np.mean(car_link_tt[10]),2)*k) + ' ' + str(round(np.mean(car_link_tt[11]),2)*k)

    s_truck = ''
    for i in range(6):
        s_truck += str(round(np.mean(truck_link_tt[i]),2)*k) + ' '
    s_truck += str(round(np.mean(truck_link_tt[6]) + np.mean(truck_link_tt[7]),2)*k) + ' '
    s_truck += str(round(np.mean(truck_link_tt[8]) + np.mean(truck_link_tt[9]),2)*k) + ' '
    s_truck += str(round(np.mean(truck_link_tt[10]),2)*k) + ' ' + str(round(np.mean(truck_link_tt[11]),2)*k)

    print('Average travel times between nodes:')
    print(s_car)
    print(s_truck)
    
    
    # Waiting time at intersections
    waitting_time = mcdta.get_waiting_time_at_intersections()
    print('Average waiting times at end of links:')
    print([round(a*3, 3) for a in waitting_time])


    # Check if spill back
    spill_back = mcdta.get_link_spillback()
    print('If link spill back:')
    print(spill_back[:12])
    
    
    # Travel times to important locations
    # Downtown: 100123 -> 150120
    path_nodes1 = [3763, 3535, 1093, 178, 5862, 7169, 6798, 1726, 6040, 5703, 8236, 7423, 2827, 5711, 2864, 7585, 
                   5597, 7841, 7596, 6251, 5911, 1430, 4822, 2769, 4318, 1632, 386]
    path_links1 = [7174, 7186, 7189, 1004121, 4120, 4118, 4068, 4072, 3960, 3958, 3957, 3941, 3938, 3929, 2027, 
                   2030, 1997, 2000, 1002040, 2039, 1002043, 2047, 2056, 2163, 2148, 2139]
    # [100123 3763 3535 1093 178 5862 7169 3 5151 4025 2618 7509 341 7915 3033 3525 6024 5320 
    #  5915 6313 5062 4785 477 1119 4739 8287 4822 2769 4318 1632 6735 150120]
    tt1_car = round(np.sum(mcdta.get_path_tt_car(path_links1, mid_intervals)),2)
    tt1_truck = round(np.sum(mcdta.get_path_tt_truck(path_links1, mid_intervals)),2)

    # Airport: 100123 -> 150194
    path_nodes2 = [3763, 3535, 4083, 2370, 5755, 5788, 5215, 6003, 1213, 2187, 2335, 3820, 8321, 6596, 5907, 587, 
                   1677, 6960, 5319, 5654, 8223, 2477, 324, 6036, 5157]
    path_links2 = [7174, 1007167, 1007149, 7150, 7200, 1007203, 7202, 1007146, 7147, 1006711, 6710, 6723, 6891, 
                   7125, 1007103, 1007098, 7100, 7110, 7107, 16554, 16804, 16807, 1016546, 16545]
    # [100123 3763 3535 4083 2370 5755 5788 5215 6003 3963 6414 7543 7763 3820 8321 6596 5907 
    #  587 1677 5569 5769 7846 666 8389 4943 8134 5865 8080 150194]
    # [100123 3763 3535 4083 2370 5755 5788 5215 6003 3963 6414 7543 7763 3820 8321 6596 11 
    #  392 6818 1737 6147 4131 2584 7604 3934 7880 6704 7426 1466 4837 647 4439 3142 2454 150194]
    tt2_car = round(np.sum(mcdta.get_path_tt_car(path_links2, mid_intervals)),2)
    tt2_truck = round(np.sum(mcdta.get_path_tt_truck(path_links2, mid_intervals)),2)

    # I-79 / I-76 interchange: 100123 -> 150275
    path_nodes3 = [3763, 3535, 1093, 178, 2224, 194, 1931, 2565, 4610, 2781, 338, 4116, 634, 3950, 4804, 3050, 
                   4096, 5801, 5027]
    path_links3 = [7174, 7186, 7189, 4151, 1007356, 7357, 1007364, 7366, 1016376, 16377, 16385, 16399, 16400, 
                   7642, 7645, 7646, 7729, 1007704]
    tt3_car = round(np.sum(mcdta.get_path_tt_car(path_links3, mid_intervals)),2)
    tt3_truck = round(np.sum(mcdta.get_path_tt_truck(path_links3, mid_intervals)),2)

    print('Travel times to important locations:')
    print(tt1_car*6, tt2_car*12, tt3_car*12, tt1_truck*6, tt2_truck*12, tt3_truck*12)

------------------------------------1-------------------------------------------
Init simulation
Average travel times between nodes:
30.0 67.26 25.44 27.24 20.04 26.82 29.7 23.22 15.42 26.34
16.68 70.86 11.16 19.68 25.44 25.8 23.7 20.52 11.28 29.4
Average waiting times at end of links:
[15.577, 21.642, 15.003, 21.502, 16.019, 15.0, 16.695, 16.623, 16.736, 16.673, 16.776, 23.131, 16.106, 20.179, 21.76, 15.254, 15.297, 16.723, 19.425, 18.209]
If link spill back:
[0 1 0 1 0 0 1 1 1 1 0 1]
Travel times to important locations:
(1390.68, 1348.92, 1473.96, 1216.02, 1316.16, 1383.3600000000001)
------------------------------------2-------------------------------------------
Init simulation
Average travel times between nodes:
29.4 84.66 24.3 41.34 20.82 25.2 56.46 30.54 27.6 15.3
19.86 73.32 17.76 23.16 30.78 23.28 28.44 26.28 15.6 14.7
Average waiting times at end of links:
[15.319, 22.879, 16.09, 25.213, 19.641, 15.787, 21.756, 18.663, 15.062, 15.466, 23.68, 18.451, 15.266, 23.451, 24.502, 15

## Find path link ids from node ids

In [7]:
graph_file_name = os.path.join('..', 'data', 'input_files_MckeesRocks_SPC_scenario_0', 'Snap_graph')

node_link_map = {}
with open(graph_file_name, 'r') as infile:
    for line in infile.readlines():
        if line[0] != '#':
            data = line.rstrip('\n').split(' ')
            node_link_map[data[1]+' '+data[2]] = data[0]

# Downtown: 100123 -> 150120
path_nodes1 = [3763, 3535, 1093, 178, 5862, 7169, 6798, 1726, 6040, 5703, 8236, 7423, 2827, 5711, 2864, 7585, 
 5597, 7841, 7596, 6251, 5911, 1430, 4822, 2769, 4318, 1632, 386]
s = ''
for i in range(len(path_nodes1) - 1):
    s += node_link_map[str(path_nodes1[i])+' '+str(path_nodes1[i+1])] + ', '
s = s[:-2]
print(s)


# Airport: 100123 -> 150194
path_nodes2 = [3763, 3535, 4083, 2370, 5755, 5788, 5215, 6003, 1213, 2187, 2335, 3820, 8321, 6596, 5907, 587, 
 1677, 6960, 5319, 5654, 8223, 2477, 324, 6036, 5157]
s = ''
for i in range(len(path_nodes2) - 1):
    s += node_link_map[str(path_nodes2[i])+' '+str(path_nodes2[i+1])] + ', '
s = s[:-2]
print(s)

# I-79 / I-76 interchange: 100123 -> 150275
path_nodes3 = [3763, 3535, 1093, 178, 2224, 194, 1931, 2565, 4610, 2781, 338, 4116, 634, 3950, 4804, 3050, 
 4096, 5801, 5027]
s = ''
for i in range(len(path_nodes3) - 1):
    s += node_link_map[str(path_nodes3[i])+' '+str(path_nodes3[i+1])] + ', '
s = s[:-2]
print(s)

7174, 7186, 7189, 1004121, 4120, 4118, 4068, 4072, 3960, 3958, 3957, 3941, 3938, 3929, 2027, 2030, 1997, 2000, 1002040, 2039, 1002043, 2047, 2056, 2163, 2148, 2139
7174, 1007167, 1007149, 7150, 7200, 1007203, 7202, 1007146, 7147, 1006711, 6710, 6723, 6891, 7125, 1007103, 1007098, 7100, 7110, 7107, 16554, 16804, 16807, 1016546, 16545
7174, 7186, 7189, 4151, 1007356, 7357, 1007364, 7366, 1016376, 16377, 16385, 16399, 16400, 7642, 7645, 7646, 7729, 1007704
